In [ ]:
!pip install language_tool_python
import language_tool_python

In [ ]:
# Создаем экземпляр инструмента для русского языка
tool = language_tool_python.LanguageTool('ru-RU')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp1raprool.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


In [ ]:
# Функция для исправления грамматических и орфографических ошибок в предложении
def correct_grammar_and_spelling(sentence):
    matches = tool.check(sentence)
    corrected_sentence = sentence
    for match in reversed(matches):  # Обрабатываем ошибки в обратном порядке
        if match.replacements:  # Проверяем, есть ли замены для ошибки
            start = match.offset
            end = match.offset + match.errorLength
            corrected_sentence = corrected_sentence[:start] + match.replacements[0]\
             + corrected_sentence[end:]
    return corrected_sentence

In [ ]:
# Функция для исправления ошибок капитализации
def correct_capitalization(text):
    # Разбиваем текст на предложения
    sentences = text.split('. ')
    corrected_sentences = []
    for sentence in sentences:
        # Исправляем первую букву предложения
        corrected_sentence = sentence[:1].upper() + sentence[1:] if sentence else sentence
        corrected_sentences.append(corrected_sentence)
    return '. '.join(corrected_sentences)


In [ ]:
# Функция для обработки текста из файла
def process_text_file(input_file, output_file):
    try:
        # Открываем файл для чтения
        with open(input_file, 'r', encoding='utf-8') as file:
            text = file.read()

        # Корректируем текст
        corrected_sentences = []
        for sentence in text.split('\n'):
            corrected_sentence = correct_grammar_and_spelling(sentence)
            corrected_sentences.append(corrected_sentence)

        # Объединяем предложения в текст
        corrected_text = '\n'.join(corrected_sentences)

        # Исправляем капитализацию в тексте
        fully_corrected_text = correct_capitalization(corrected_text)

        # Сохраняем исправленный текст в новый файл
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(fully_corrected_text)

        print(f"Обработка завершена. Исправленный текст сохранен в файл: {output_file}")

    except Exception as e:
        print(f"Ошибка при обработке файла: {e}")

In [ ]:
# Пример использования
input_file = 'input.txt'  # Имя входного файла
output_file = 'corrected_output.txt'  # Имя выходного файла

# Загрузка, обработка и сохранение текста
process_text_file(input_file, output_file)

Обработка завершена. Исправленный текст сохранен в файл: corrected_output.txt


In [ ]:
import pandas as pd

def compare_files_and_calculate_metrics(input_file, corrected_file, reference_file):
    """
    Сравнивает три файла (исходный, исправленный и контрольный) и вычисляет Precision, Recall и F1-Score.
    Также формирует датафрейм с детализацией ошибок на уровне слов.

    :param input_file: Исходный файл с ошибками.
    :param corrected_file: Файл с исправлениями, полученный от алгоритма.
    :param reference_file: Контрольный файл без ошибок.
    :return: Метрики Precision, Recall, F1-Score и датафрейм с детализацией ошибок.
    """
    try:
        # Чтение файлов
        with open(input_file, 'r', encoding='utf-8') as f:
            input_text = f.read().splitlines()

        with open(corrected_file, 'r', encoding='utf-8') as f:
            corrected_text = f.read().splitlines()

        with open(reference_file, 'r', encoding='utf-8') as f:
            reference_text = f.read().splitlines()

        # Проверка длины файлов
        if not (len(input_text) == len(corrected_text) == len(reference_text)):
            raise ValueError("Файлы должны содержать одинаковое количество строк для корректного сравнения.")

        # Инициализация счетчиков
        true_positive = 0  # Правильно исправленные ошибки
        false_positive = 0  # Ложные исправления (там, где не было ошибки)
        false_negative = 0  # Пропущенные ошибки

        # Список для формирования датафрейма
        error_details = []

        # Сравнение строк на уровне слов
        for i in range(len(input_text)):
            input_words = input_text[i].split()
            corrected_words = corrected_text[i].split()
            reference_words = reference_text[i].split()

            max_len = max(len(input_words), len(corrected_words), len(reference_words))

            for j in range(max_len):
                input_word = input_words[j] if j < len(input_words) else ''
                corrected_word = corrected_words[j] if j < len(corrected_words) else ''
                reference_word = reference_words[j] if j < len(reference_words) else ''

                # Если ошибка исправлена правильно
                if input_word != reference_word and corrected_word == reference_word:
                    true_positive += 1
                    error_details.append({
                        'Исходное слово': input_word,
                        'Исправленное слово': corrected_word,
                        'Ожидаемое слово': reference_word,
                        'Статус': 'True Positive'
                    })
                # Если алгоритм внёс исправление, но ошибки не было
                elif input_word == reference_word and corrected_word != reference_word:
                    false_positive += 1
                    error_details.append({
                        'Исходное слово': input_word,
                        'Исправленное слово': corrected_word,
                        'Ожидаемое слово': reference_word,
                        'Статус': 'False Positive'
                    })
                # Если ошибка не исправлена
                elif input_word != reference_word and corrected_word != reference_word:
                    false_negative += 1
                    error_details.append({
                        'Исходное слово': input_word,
                        'Исправленное слово': corrected_word,
                        'Ожидаемое слово': reference_word,
                        'Статус': 'False Negative'
                    })

        # Вычисление метрик
        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Создание датафрейма
        error_df = pd.DataFrame(error_details)

        return precision, recall, f1_score, error_df

    except Exception as e:
        print(f"Ошибка при вычислении метрик: {e}")
        return 0, 0, 0, pd.DataFrame()

# Пример использования
input_file = 'input.txt'  # Исходный файл с ошибками
corrected_file = 'corrected_output.txt'  # Файл с исправлениями
reference_file = 'reference.txt'  # Контрольный файл без ошибок

precision, recall, f1_score, error_df = compare_files_and_calculate_metrics(input_file, corrected_file, reference_file)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

# Вывод датафрейма
if not error_df.empty:
    print("\nДетализация ошибок:")
    print(error_df)


Precision: 1.00
Recall: 0.84
F1 Score: 0.91

Детализация ошибок:
        Исходное слово  Исправленное слово     Ожидаемое слово          Статус
0          беспилотные         Беспилотные         Беспилотные   True Positive
1                (бас)               (бас)               (БАС)  False Negative
2              нахотят             находят             находят   True Positive
3           приминение          применение          применение   True Positive
4        используються        используются        используются   True Positive
5            например,           Например,           Например,   True Positive
6               помощю              помощь             помощью  False Negative
7               бистро              бистро              быстро  False Negative
8        экалогических       экологических       экологических   True Positive
9                также               Также               Также   True Positive
10         приминяются         применяются         применяются   T